## This file contains SQL queries for 

#### Data clecning pipeline and creating VALID_PARTICIPANTS and VALID_TEST_SECTIONS tables

#### Filtering valid participants and their test sections (first 15) (if necessary)
#### Sampling valid participants and their test sections (RANDOM via SQL and stratified via python)


In [1]:
from configparser import ConfigParser
from sqlalchemy import create_engine
import pandas as pd
from geolite2 import geolite2
import geopandas as gpd
import pycountry
import re
import seaborn as sns
import matplotlib.pyplot as plt
get_ipython().magic('matplotlib inline')
sns.set(color_codes=True)
%load_ext autotime

def read_db_config(filename='config-dump.ini', section='mysql'):
    """ Read database configuration file and return a dictionary object
    :param filename: name of the configuration file
    :param section: section of database configuration
    :return: a dictionary of database parameters
    """
    # create parser and read ini configuration file
    parser = ConfigParser()
    parser.read(filename)
 
    # get section, default to mysql
    db = {}
    if parser.has_section(section):
        items = parser.items(section)
        for item in items:
            db[item[0]] = item[1]
    else:
        raise Exception('{0} not found in the {1} file'.format(section, filename))
 
    return db

dbconfig = read_db_config()
CONN = 'mysql+mysqlconnector://'+dbconfig['user']+':'+dbconfig['pwd']+'@'+dbconfig['host']+':'+dbconfig['port'] + '/'+dbconfig['dbname']
engine = create_engine(CONN)

In [ ]:
# 2018-09-20 2:14pm: Public release date
# Local timestamp ms: 1537441800000

# SELECT MIN(TEST_SECTION_ID)
# FROM KEYSTROKES
# WHERE PRESS_TIME > 1537441800000;

# The first test section recorded after this timestamp is TEST_SECTION_ID =2458
# and correponds with the PARTICIPANT_ID = 607

In [ ]:
## Run the following in the database (order matters)

# 0. Compute IKI for each record in KEYSTROKES (~ 16 mins)
"""
ALTER TABLE KEYSTROKES ADD COLUMN IKI DOUBLE;

select @prev:=PRESS_TIME from KEYSTROKES 
ORDER BY PRESS_TIME ASC, TEST_SECTION_ID ASC
LIMIT 1,1;

update KEYSTROKES k
set k.IKI = ( 
    select pk.IKI 
    from ( 
        select 
            ks.KEYSTROKE_ID, 
            ks.PRESS_TIME, 
            @prev, 
            ks.PRESS_TIME - @prev as IKI,
            @prev:=ks.PRESS_TIME
        from KEYSTROKES ks
        ORDER BY ks.PRESS_TIME ASC, ks.TEST_SECTION_ID ASC
    ) as pk
    where 
        k.KEYSTROKE_ID = pk.KEYSTROKE_ID 
)
"""

# 1.a. Define and select valid participants 
"""
CREATE TABLE tmp LIKE PARTICIPANTS;
INSERT INTO tmp 
    SELECT * FROM PARTICIPANTS
    WHERE AGE IS NOT NULL
    AND WPM > 0 
    AND WPM < 200 
    AND ERROR_RATE < 25 
    AND DEVICE ='mobile' 
    AND PARTICIPANT_ID > 607
"""
# 1.b. Calculate 2SD from mean age. 
"""
SELECT 
   @Std := 2*STD(age), 
   @Avg := AVG(age) 
FROM tmp
"""
# 1.c. Use only data with age within 2SD from the mean. 
"""
CREATE TABLE VALID_PARTICIPANTS LIKE PARTICIPANTS;
INSERT INTO VALID_PARTICIPANTS 
    SELECT 
       * 
    FROM 
       tmp 
    WHERE 
        AGE > 5 AND AGE < @Avg + @Std
"""


# 2. Select test sections from valid participants EXCLUDING test sections with IKI > 5000
# Select only data with non-negative WPM and not-null USER_INPUT
"""
CREATE TABLE VALID_TEST_SECTIONS LIKE TEST_SECTIONS;
INSERT INTO VALID_TEST_SECTIONS
    SELECT * FROM TEST_SECTIONS
    WHERE PARTICIPANT_ID IN (
        SELECT PARTICIPANT_ID FROM VALID_PARTICIPANTS
    ) 
    AND WPM > 0
    AND WPM < 200 
    AND ERROR_RATE < 25
    AND USER_INPUT IS NOT NULL
    AND TEST_SECTION_ID NOT IN (
        SELECT DISTINCT TEST_SECTION_ID
        FROM KEYSTROKES
        WHERE IKI > 5000
    )
"""

# 3. Make sure there are at least 15 test sections for each valid participant
# If this query returns non-zero number of rows, perform steps 4 and 5.
"""
SELECT * 
FROM (
SELECT PARTICIPANT_ID, COUNT(TEST_SECTION_ID) AS TS_COUNT
FROM VALID_TEST_SECTIONS
GROUP BY PARTICIPANT_ID
ORDER BY TS_COUNT ASC) AS tmp
WHERE tmp.TS_COUNT < 15
"""

# 4. Update VALID_PARTICIPANTS to discard those with less than 15 valid test sections
"""
DROP TABLE VALID_PARTICIPANTS;
CREATE TABLE VALID_PARTICIPANTS LIKE PARTICIPANTS;
INSERT INTO VALID_PARTICIPANTS 
    SELECT * FROM PARTICIPANTS
    WHERE PARTICIPANT_ID IN (
        SELECT tmp.PARTICIPANT_ID 
            FROM (
            SELECT PARTICIPANT_ID, COUNT(TEST_SECTION_ID) AS TS_COUNT
            FROM VALID_TEST_SECTIONS
            
            GROUP BY PARTICIPANT_ID
            ORDER BY TS_COUNT ASC) AS tmp
            WHERE tmp.TS_COUNT >= 15
    );
"""

# 5. Update VALID_TEST_SECTIONS to match VALID_PARTICIPANTS
"""
DROP TABLE VALID_TEST_SECTIONS;
CREATE TABLE VALID_TEST_SECTIONS LIKE TEST_SECTIONS;
INSERT INTO VALID_TEST_SECTIONS
    SELECT * FROM TEST_SECTIONS
    WHERE PARTICIPANT_ID IN (
        SELECT PARTICIPANT_ID FROM VALID_PARTICIPANTS  
    ) 
    AND WPM > 0
    AND WPM < 200 
    AND ERROR_RATE < 25
    AND USER_INPUT IS NOT NULL
    AND TEST_SECTION_ID NOT IN (
        SELECT DISTINCT TEST_SECTION_ID
        FROM KEYSTROKES
        WHERE IKI > 5000
    )
"""

# 6. Double-check step 3. Should return zero rows now. Remove tmp.

# 7. Include only first 15 sentences from valid participants.
"""
#RENAME TABLE VALID_TEST_SECTIONS TO tmp;

CREATE TABLE FIRST_15_TEST_SECTIONS LIKE VALID_TEST_SECTIONS;
INSERT INTO FIRST_15_TEST_SECTIONS
    SELECT * FROM VALID_TEST_SECTIONS
    WHERE TEST_SECTION_ID IN (
        SELECT TEST_SECTION_ID
            FROM (
                  SELECT  @type_number := IF(@PARTICIPANT_ID = PARTICIPANT_ID, @type_number + 1, 1) AS type_number, 
                          @PARTICIPANT_ID := PARTICIPANT_ID as PARTICIPANT_ID, 
                          TEST_SECTION_ID
                  FROM 
                      (SELECT @type_number := 1) x, 
                      (SELECT TEST_SECTION_ID, @PARTICIPANT_ID := PARTICIPANT_ID as PARTICIPANT_ID FROM VALID_TEST_SECTIONS WHERE TS_IKI < 5000 ORDER BY PARTICIPANT_ID) y
                ) z
            WHERE type_number <= 15
    );  
"""



## By now, the data for analysis are:
#### VALID_PARTICIPANTS
#### VALID_TEST_SECTIONS 
#### FIRST_15_TEST_SECTIONS
#### + Original raw data

# Other snippets: queries and drafts

## Filter VALID_KEYSTROKES

In [ ]:
# Select KEYSTROKES associated with VALID_TEST_SECTIONS
# ~ 6 minutes, 7 586 255 rows in the table
"""
CREATE TABLE VALID_KEYSTROKES LIKE KEYSTROKES;
ALTER TABLE VALID_KEYSTROKES DROP COLUMN IKI;
INSERT INTO VALID_KEYSTROKES
    SELECT KEYSTROKE_ID, PRESS_TIME, RELEASE_TIME, LETTER, TEST_SECTION_ID, KEYCODE FROM KEYSTROKES
    WHERE TEST_SECTION_ID IN (
        SELECT TEST_SECTION_ID FROM VALID_TEST_SECTIONS
    )
    
"""

"""
ALTER TABLE VALID_KEYSTROKES ADD IKI INT AS (RELEASE_TIME-PRESS_TIME)
"""

## Filter VALID_LOG_DATA

In [ ]:
# Select (some columns from) LOG_DATA associated with VALID_TEST_SECTIONS
# ~ 15 minutes, 22 829 177 rows in the table
"""
CREATE TABLE VALID_LOG_DATA LIKE LOG_DATA;
ALTER TABLE VALID_LOG_DATA DROP COLUMN DEVICE_ORIENTATION;
ALTER TABLE VALID_LOG_DATA DROP COLUMN INPUT;
ALTER TABLE VALID_LOG_DATA DROP COLUMN PRESSED;
INSERT INTO VALID_LOG_DATA
    SELECT LOG_DATA.LOG_DATA_ID, LOG_DATA.TEST_SECTION_ID, LOG_DATA.TYPE, LOG_DATA.KEY, LOG_DATA.CODE, LOG_DATA.DATA, LOG_DATA.TIMESTAMP, LOG_DATA.SCREEN_ORIENTATION FROM LOG_DATA
    WHERE LOG_DATA.TEST_SECTION_ID IN (
        SELECT TEST_SECTION_ID FROM VALID_TEST_SECTIONS
    )
"""

## Clean VALID_LOG_DATA

In [ ]:
# Make presentation nicer by replacing undefined and Unidentified with -1
# DATA: 14:10 -> 14:25
# KEY: 14:30 -> 14:36
# CODE: 14:37 -> 14:42

"""
UPDATE VALID_LOG_DATA
SET DATA = CASE WHEN DATA = 'undefined' THEN '-1' END
WHERE DATA in ('undefined');

UPDATE VALID_LOG_DATA
SET `KEY` = CASE WHEN `KEY` = 'undefined' THEN '-1' 
                 WHEN `KEY` = 'Unidentified' THEN '-2' END
WHERE `KEY` in ('undefined', 'Unidentified');

UPDATE VALID_LOG_DATA
SET `CODE` = CASE WHEN `CODE` = 'undefined' THEN '-1' END
WHERE `CODE` in ('undefined');
"""

## Join VALID_LOG_DATA with VALID_KEYSTROKES

In [ ]:
# Join VALID_LOG_DATA and VALID_KEYSTROKES 
# 15:27 -> 15:59

"""
CREATE TABLE VALID_JOIN AS 
(
    SELECT
        VALID_LOG_DATA.TIMESTAMP,
        VALID_KEYSTROKES.PRESS_TIME,
        VALID_KEYSTROKES.RELEASE_TIME,
        VALID_LOG_DATA.TYPE,
        VALID_LOG_DATA.KEY,
        VALID_LOG_DATA.CODE,
        VALID_LOG_DATA.DATA,
        VALID_KEYSTROKES.LETTER,
        VALID_KEYSTROKES.KEYCODE,
        VALID_LOG_DATA.TEST_SECTION_ID,
        VALID_LOG_DATA.SCREEN_ORIENTATION,
        VALID_LOG_DATA.LOG_DATA_ID,
        VALID_KEYSTROKES.KEYSTROKE_ID
        
    FROM
        VALID_LOG_DATA 
        LEFT JOIN VALID_KEYSTROKES 
            ON VALID_LOG_DATA.TEST_SECTION_ID = VALID_KEYSTROKES.TEST_SECTION_ID
            AND VALID_LOG_DATA.TIMESTAMP = VALID_KEYSTROKES.PRESS_TIME
);
"""
    
# Seems like keyup events match about correctly with RELEASE_TIME thus may be omitted? 

In [ ]:
# Compute  KPD (Key press duration) for each record in VALID_KEYSTROKES 
"""
ALTER TABLE VALID_KEYSTROKES ADD KPD INT AS (RELEASE_TIME-PRESS_TIME)
"""

In [ ]:
# Find and invalidate participants appeared to mark (almost) all-first in the questionnaire: 
# Seems like there are only few such participants (<10)
"""
SELECT * 
FROM PARTICIPANTS
WHERE 
    GENDER = 'none' 
    AND (TYPE_TEST_LANG = 'always' OR TYPE_TEST_LANG = '') 
    AND FINGERS = 'left_hand;thumbs' 
    AND HAS_TAKEN_TYPING_COURSE = 1
    AND KEYBOARD_TYPE = 'mobile'
    AND LAYOUT = 'qwerty'
"""

In [ ]:
# Get percentiles 

"""
use typing;
SELECT
    participant_id,
    @prev := @curr as prev,
    @curr := age as curr,
    @rank := IF(@prev > @curr, @rank+@ties, @rank) AS rank,
    @ties := IF(@prev = @curr, @ties+1, 1) AS ties,
    (1-@rank/@total) as percentrank
FROM
    participants,
    (SELECT
        @curr := null,
        @prev := null,
        @rank := 0,
        @ties := 1,
        @total := count(*) from participants where age is not null
    ) b
WHERE
    age is not null
ORDER BY
    age DESC
"""

In [ ]:
# Select participants whoode age is within 2 standard deviations from the mean 
"""
SELECT 
   @Std := 2*STD(age), 
   @Avg := AVG(age) 
FROM VALID_PARTICIPANTS

SELECT 
   * 
FROM 
   VALID_PARTICIPANTS 
WHERE 
   AGE > @Avg - @Std AND AGE < @Avg + @Std
"""

#### Find IKI-invalid test sections

In [ ]:


# Here are some invalid keystrokes (156 rows)
"""
SELECT KEYSTROKE_ID, TEST_SECTION_ID, IKI FROM VALID_KEYSTROKES
WHERE IKI > 5000
"""
# or (72 distinct test_sections)
"""
SELECT TEST_SECTION_ID, count(IKI) FROM VALID_KEYSTROKES
WHERE IKI > 5000
GROUP BY TEST_SECTION_ID
"""

In [ ]:
# Find TEST_SECTION_ID's for which IKI > 5000
"""
SELECT DISTINCT TEST_SECTION_ID
FROM KEYSTROKES
WHERE IKI > 5000
"""

In [ ]:
# https://rextester.com/IUPWT52140

# To select only 15 test sections for the analysis,
# find TEST_SECTION_ID range for each participant and use only first 15.
# 200797 total (after steps 1-6 above)
# 156720 filtered (using lines 12-21 below)
"""
RENAME TABLE VALID_TEST_SECTIONS TO tmp;
CREATE TABLE VALID_TEST_SECTIONS LIKE TEST_SECTIONS;
INSERT INTO VALID_TEST_SECTIONS
    SELECT * FROM TEST_SECTIONS
    WHERE TEST_SECTION_ID IN (
        SELECT TEST_SECTION_ID
            FROM (
                  SELECT  @type_number := IF(@PARTICIPANT_ID = PARTICIPANT_ID, @type_number + 1, 1) AS type_number, 
                          @PARTICIPANT_ID := PARTICIPANT_ID as PARTICIPANT_ID, 
                          TEST_SECTION_ID
                  FROM 
                      (SELECT @type_number := 1) x, 
                      (SELECT TEST_SECTION_ID, @PARTICIPANT_ID := PARTICIPANT_ID as PARTICIPANT_ID FROM tmp ORDER BY PARTICIPANT_ID) y
                ) z
            WHERE type_number <= 15
    );  
"""




In [ ]:
# 10 448 VALID_PARTICIPANTS
# By now, the number of rows in KEYSTROKES associated with VALID_TEST_SECTIONS is 7 586 255 (9 619 766 in more-than-15-ts case)
# (Total number of rows is 16 794 011)
# SELECT COUNT(KEYSTROKE_ID) FROM KEYSTROKES 
# WHERE TEST_SECTION_ID IN (
#    SELECT TEST_SECTION_ID 
#    FROM VALID_TEST_SECTIONS
# )

# Same for LOG_DATA: 22 829 177 (28 961 876) of valid data vs 51 540 020 total 

### Sampling ( in progress ... )

In [ ]:
# RANDOM sampling: 
# Create sample (1% of participants) with assosiated test sections
"""
CREATE TABLE SAMPLE_PARTICIPANTS LIKE PARTICIPANTS;
INSERT INTO SAMPLE_PARTICIPANTS 
SELECT *
FROM VALID_PARTICIPANTS 
WHERE rand() < 0.01;

CREATE TABLE SAMPLE_TEST_SECTIONS LIKE TEST_SECTIONS;
INSERT INTO SAMPLE_TEST_SECTIONS 
SELECT *
FROM VALID_TEST_SECTIONS 
WHERE VALID_TEST_SECTIONS.PARTICIPANT_ID IN (
    SELECT PARTICIPANT_ID FROM SAMPLE_PARTICIPANTS
);
"""


In [8]:
# Sampling 10% (1044 people) from participants with 50% male, 50% female (522 male, 522 female)
# TODO AND language?

P_QUERY = "SELECT * FROM VALID_PARTICIPANTS" 
ps = pd.read_sql_query(P_QUERY, engine)
ps.fillna(0, inplace=True)
ps.describe()

PARTICIPANT_ID     SCREEN_W     SCREEN_H          AGE  \
count     4873.000000  4873.000000  4873.000000  4873.000000   
mean     48459.319721   425.313359   700.774677    24.979684   
std      26127.281203   163.177424    94.362041    10.223558   
min        629.000000   320.000000   320.000000     6.000000   
25%      25271.000000   360.000000   640.000000    19.000000   
50%      49383.000000   375.000000   667.000000    23.000000   
75%      73819.000000   414.000000   736.000000    29.000000   
max      87241.000000  1112.000000  1031.000000   120.000000   

       HAS_TAKEN_TYPING_COURSE          WPM   ERROR_RATE  
count              4873.000000  4873.000000  4873.000000  
mean                  0.269444    34.926684     2.768454  
std                   0.443716    13.040143     2.692242  
min                   0.000000     0.303683     0.000000  
25%                   0.000000    25.874385     1.047904  
50%                   0.000000    33.097760     1.917404  
75%                   1.000000    42.355041     3.582555  
max                   1.000000   175.998524    24.842767

time: 485 ms


In [ ]:
# 